# Random Forest

Random forest is a supervised learning algorithm. The "forest" it builds, is an ensemble of decision trees. Decision trees classify the data points by sorting them down the tree from the root to some leaf, with the leaf node providing the classification of the example. The feature that we decide to split on is the feature that gives us the most information. Each node in the tree acts as a test case for some attribute, and each edge descending from the node corresponds to the possible answers to the test case. This process is recursive in nature and is repeated for every subtree rooted at the new node. 

In a random forest each decision tree in the forest considers a random subset of features as well as a random subset of the training data points. The makes each tree different in their own way and each tree will come to decisions using different features as well as different data points. When using a random forest to classify we run the data through every tree and then use the make a decision based on popular vote (classification) or average (regression).

In [1]:
import numpy as np
import math

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes, fetch_california_housing

from scipy.optimize import minimize

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Random Forest from scratch

In [3]:
class TreeEnsemble():
    '''
    Tree ensemble is a class that holds many Decision Trees and uses their combined decision/vote to return a prediction.
    '''
    def __init__(self, n_trees, sample_sz, min_leaf):
        '''
        n_trees is the number of trees to create
        sample_sz is the size of the sample set to use of each of the trees in the forest (chose the samples randomly, with or without repetition)
        min_leaf is the minimal number of samples in each leaf node of each tree in the forest
        '''
        self.trees = []
        self.n_trees = n_trees
        self.sample_sz = sample_sz
        self.min_leaf = min_leaf
        
    def fit(self, X, y):
        '''
        Fit/train model
        X: a matrix of data values (rows are samples, columns are attributes)
        y: a vector of corresponding target values
        '''
        for tree_i in range(self.n_trees):
            tree = DecisionTree(self.sample_sz, self.min_leaf)
            self.trees.append(tree.fit(X, y))
  
    def predict(self, X):
        '''
        Predict class labels using fitted/trained model
        X: a matrix of data values (rows are samples, columns are attributes)
        '''
        pred = []
        for tree in self.trees:
            pred.append(tree.predict(X))
        return np.asarray(pred).mean(axis=0)

    def oob_mse(self):
        '''
        Compute the mean squared error over all out of bag (oob) samples. That is, for each sample calculate the squared error using  predictions from 
        the trees that do not contain x in their respective bootstrap sample, then average this score for all samples.
        '''
        errors = []
        for tree in self.trees:
            errors.append(tree.oob_mse())
        return np.asarray(errors).mean()

In [12]:
class DecisionTree():
    '''
    A decision tree is a flowchart-like structure in which each internal node represents a question on an attribute (Taller than 1.5 meters?, Black Hair?),
    each branch represents the outcome of the question on that datapoint, and each leaf node represents a class label 
    '''
    def __init__(self, sample_sz, min_leaf):
        '''
        sample_sz: amount of data to use when fitting tree
        min_leaf: minimum amount of samples allowed in a leaf
        '''
        self.min_leaf = min_leaf
        self.sample_size =  sample_sz

    def fit(self, X, y):
        '''
        X: a matrix of data values (rows are samples, columns are attributes)
        y: a vector of corresponding target values
        '''
        # sample from X
        num_samples = X.shape[0]
        sample = np.random.randint(0, num_samples, self.sample_size)
        self.X = X[sample]
        self.y = y[sample]
        not_sampled = [i for i in np.arange(num_samples) if i not in sample]
        self.oob_X = X[not_sampled]
        self.oob_y = y[not_sampled]
        # call recursive builder
        self.top_node = Node()
        self.recursive_tree_builder(self.X, self.y, self.top_node)
        
        return self

    def predict(self, X):
        '''
        Predict using fitted/trained model
        X: a matrix of data values (rows are samples, columns are attributes)
        '''
        return np.apply_along_axis(self.predict_single, arr=X, axis=1)
    
    def predict_single(self, x):
        '''
        get prediction of a single datapoint for this tree
        X: single row of data
        '''
        node = self.top_node
        val = node.value
        feat_idx = node.feature
        while(True):
            if node.value is None or node.feature is None:
                return node.mean
            if x[feat_idx] > val:
                node = node.bigger
                val = node.value
                feat_idx = node.feature
            else:
                node = node.smaller
                val = node.value
                feat_idx = node.feature
                
    def oob_mse(self):
        '''
        Compute the mean squared error over all out of bag (oob) samples for this tree.
        '''
         return mean_squared_error(self.predict(self.oob_X), self.oob_y)
    
    def recursive_tree_builder(self, X, y, curr_node):
        '''
        X: a matrix of data values (rows are samples, columns are attributes)
        y: a vector of corresponding target values
        curr_node: current node in tree that we are working on.
        '''
        #if we have less than min leaf we return
        if X.shape[0] <= 2 * self.min_leaf:
            # update as end node with proba
            curr_node.mean = np.mean(y)
            return
        else:
            # find best feature to split by
            curr_node.feature, curr_node.value = self.best_split(X, y)
            if curr_node.value is None:
                curr_node.mean = np.mean(y)
                return
            bigger = X[:,curr_node.feature] > curr_node.value
            smaller = X[:,curr_node.feature] <= curr_node.value
            self.recursive_tree_builder(X[bigger,:], y[bigger], curr_node.set_bigger())
            self.recursive_tree_builder(X[smaller,:], y[smaller], curr_node.set_smaller())
            return
                
    def best_split(self, X, y):
        '''
        X: a matrix of data values (rows are samples, columns are attributes)
        y: a vector of corresponding target values
        '''
        # for each feature we check 'all' points and take point with lowest
        kwargs = {'y': y, 'min_leaf':self.min_leaf}
        min_split_per_feature, error = np.apply_along_axis(self.get_min_split, arr=X, axis=0, **kwargs)
        feature = np.argmin(error)
        split_val = min_split_per_feature[feature]
        return feature, split_val
                                   
    def get_min_split(self, feat, y, min_leaf):
        '''
        get best split of data
        feat: array of all of the features
        y: a vector of corresponding target values
        min_leaf: minimum amount of samples allowed in a leaf
        '''
        idxs = np.argsort(feat)
        feat = np.sort(feat)
        y = y[idxs]
        bounds = feat[min_leaf: -(min_leaf + 1)]

        min_error = math.inf
        split_val = None
        for trial in bounds:
            if self.bad_trial(trial, feat):
                pass
            error = self.get_var_error(trial, feat, y)
            if error < min_error:
                min_error = error
                split_val = trial
        return (split_val, min_error)

    def bad_trial(self, split, feat):
        '''
        This returns true if we cant split our node anymore
        split: where the feature should be split
        feat: single column of data (all datapoints for a single feature)
        '''
        bigger = feat > split
        smaller = feat <= split
        return (feat[bigger].shape[0] <= self.min_leaf) or (feat[smaller].shape[0] <= self.min_leaf)
                                
    def get_var_error(self, split, feat, y):
        '''
        split: where the feature should be split
        feat: single column of data (all datapoints for a single feature)
        y: a vector of corresponding target values
        '''
        bigger = feat > split
        smaller = feat <= split

        var_bigger = np.square(np.var(feat[bigger]))
        var_smaller = np.square(np.var(feat[smaller]))
        
        bigger_size = feat[bigger].shape[0]
        smaller_size = feat[smaller].shape[0]
        n = feat.shape[0]

        return (bigger_size/n)*var_bigger + (smaller_size/n)*var_smaller

In [11]:
class Node:
    '''
    This class represents a single node from a DecisionTree.
    '''
    def __init__(self):
        self.feature = None
        self.value = None
        self.smaller = None
        self.bigger = None
        self.mean = None

    def set_bigger(self):
        """
        Creates child, adds to child list and returns child
        """
        self.bigger = Node()
        return self.bigger
                                   
    def set_smaller(self):
        """
        Creates child, adds to child list and returns child
        """
        self.smaller = Node()
        return self.smaller

    def is_leaf(self):
        '''
        Returns true if node is leaf
        '''
        return self.smaller is None and self.bigger is None

## Predict on California Housing data

In [6]:
X, y = fetch_california_housing(return_X_y=True)

In [7]:
X.shape

(20640, 8)

In [8]:
te = TreeEnsemble(n_trees=20, sample_sz=1000, min_leaf=20)

In [9]:
te.fit(X, y)

In [10]:
te.oob_mse()

1.3244075290629032

#### This is a pretty good result but we will see that we can do even better than this with Gradient Boosting Trees